In [ ]:
# default_exp signUp

# SignUp

> Interface to sign up user

In [2]:
#hide
from nbdev.showdoc import *
import src.setEnv 

In [13]:
#export 
from src.userTable import UserTable
from awsSchema.apigateway import Response, Event
from copy import deepcopy
from beartype import beartype
from uuid import uuid4
from src.saltHashPassword import hash_password, check_password
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from time import time
import json

In [4]:
#check whether hash password is working here
examplePassword1 = '123'
exampleHashedPassword1 = hash_password(examplePassword1)
exampleHashedPassword1

'6110775ef853857efe6074bb0aee9ae9df230d556cbd235012f5fb0ec4eb55c4:020b227ca5ea415990b49f409415ee73'

## User Input Class

In [5]:
#export 
@dataclass_json
@dataclass
class UserInput:
    username: str
    password: str
    
    @property
    def passwordHash(self):
        return hash_password(self.password)
    
    def saveTable(self):
        passwordHashed = self.passwordHash
        userTable = UserTable(
                userId = str(uuid4()),
                username = self.username,
                date = time(),
                passwordHash = passwordHashed
            )
        userTable.save()
        

In [6]:
example = UserInput.from_dict({'username': 'DelG', 'password': '123'})
example

UserInput(username='DelG', password='123')

## Main Lambda Handler

In [7]:
#export 
class H:
    class ParseInputError(Exception): pass
    class SavingError(Exception): pass
    @classmethod
    @beartype
    def parseInput(cls, event:dict)->UserInput:
        try:
            user = Event.parseDataClass(UserInput, deepcopy(event))
            return user
        except Exception as e:
            raise cls.ParseInputError(e)
    
    @classmethod
    @beartype
    def saveUserMethod(cls, user:UserInput)->bool:
        try:
            user.saveTable()
            return True
        except Exception as e:
            raise cls.SavingError(e)
    
        

In [8]:
#export
def signUp(event, *args):
    try:
        user = H.parseInput(event)
        H.saveUserMethod(user)
        return Response.returnSuccess()
    except H.SavingError as e:
        return Response.returnError(f'failed saving user {e}')
    except Exception as e:
        return Response.returnError(f' unknown error {e}')
    

In [9]:
input_ = {'username': 'Delgg', 'password': '1276'}
event = Event.getInput(body = input_)
signUp(event)

Delgg


{'body': '{}',
 'statusCode': 200,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

In [10]:
epoch = time()
epoch

1660574887.723819

In [14]:
a = Event.getInput({'username': 'DelG196', 'password': '12345'})
json.dumps(a)

'{"body": "{\\"username\\":\\"DelG196\\",\\"password\\":\\"12345\\"}", "httpMethod": null, "multiValueHeaders": null, "multiValueQueryStringParameters": null, "path": null, "pathParameters": null, "queryStringParameters": {}, "requestContext": null, "headers": {}, "statusCode": 200, "isBase64Encoded": false}'

In [23]:
a = 'very interesting'
if a:
    print('true')
else:
    print('false')

true
